In [166]:
# If you what use ngrams.py you should use python2
# Or, otherwise, you need to modify ngrams.py by yourself in order to use it in python3.
from __future__ import print_function
from __future__ import unicode_literals

import re
import numpy as np
from ngrams import ngrams
from collections import defaultdict
from math import log, pow, inf

In [2]:
small_corpus = ['Why dont we start here',
                  'Why dont we end there',
                  'Let us start with a few other examples',
                  'We never start with an example with so few tokens',
                  'Tokens can be words that we start with in example docs'
                  ]

# HW3: Language Modeling
For this part of the assignment, you will implement two simple count-based n-gram language models: one based on maximum-likelihood estimation, and another based on Witten-Bell smoothing. The data you will be using is a subset of the Penn Treebank's tagged Wall Street Journal articles on which we have done some initial processing. There are two versions of the data for this assignment:

##### wsj.pos.gz
##### wsj-normalized.pos.gz
The difference is that, in the second (normalized) version of the data, we have collapsed some entries from certain tag categories (e.g. CDs, NNPs, etc.) into type-tokens to help reduce sparsity. Take a look at the data and see for yourself. Consider: what would be the benefits and drawbacks to this method of sparsity reduction? Note that, for this part of the assignment, the tags are un-necessary, so you'll want to work with the un-normalized version of the corpus.

https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

### Task 1: produce a tag-free corpus

For this task, you have two jobs. 
* First, you need to write a function to filter out all tags. 
* Second, Make sure your code works for both wsj.pos.gz and wsj-normalized.pos.gz

####What to turn in
* your code
* some samples to show me that your code works as it should be

In [188]:
import gzip

#regex = re.compile(r"/[A-Z$,.]{1,4}") #stackoverflow says unnecessary, cached anyway
maxln = 7

def rem_tag(line):
    line = line.decode('utf-8')
    return re.sub(r"/[A-Z$,.`']{1,4}",'',line)

print("######################## WSJ.POS Examples########################")
with gzip.open('wsj.pos.gz','r') as wsj:
    count = 0
    for line in wsj:
        print(line)
        print("\n")
        line = rem_tag(line)
        print(line)
        count += 1
        if count > maxln:
            break

print("######################## WSJ.NORMALIZED Examples:########################")
with gzip.open('wsj-normalized.pos.gz','r') as wsj:
    count = 0
    for line in wsj:
        print(line)
        print("\n")
        line = rem_tag(line)
        print(line)
        count += 1
        if count > maxln:
            break


######################## WSJ.POS Examples########################
b'Digital/NNP Equipment/NNP Corp./NNP reported/VBD a/DT 32/CD %/NN decline/NN in/IN net/JJ income/NN on/IN a/DT modest/JJ revenue/NN gain/NN in/IN its/PRP$ fiscal/JJ first/JJ quarter/NN ,/, causing/VBG some/DT analysts/NNS to/TO predict/VB weaker/JJR results/NNS ahead/RB than/IN they/PRP had/VBD expected/VBN ./.\n'


Digital Equipment Corp. reported a 32 % decline in net income on a modest revenue gain in its fiscal first quarter , causing some analysts to predict weaker results ahead than they had expected .

b'Although/IN the/DT second-largest/JJ computer/NN maker/NN had/VBD prepared/VBN Wall/NNP Street/NNP for/IN a/DT poor/JJ quarter/NN ,/, analysts/NNS said/VBD they/PRP were/VBD troubled/VBN by/IN signs/NNS of/IN flat/JJ U.S./NNP orders/NNS and/CC a/DT slowdown/NN in/IN the/DT rate/NN of/IN gain/NN in/IN foreign/JJ orders/NNS ./. The/DT Maynard/NNP ,/, Mass./NNP ,/, company/NN is/VBZ in/IN a/DT transition/NN in/IN wh

<font color="red">Self assessment:</font>

In [229]:
#Constructing Corpus
wsj_corpus = []
wsj_norm_corpus = []
with gzip.open('wsj.pos.gz','r') as wsj:
    for line in wsj:
        line = rem_tag(line)
        wsj_corpus += line.replace(',','').split(' .')

with gzip.open('wsj-normalized.pos.gz','r') as wsj:
    for line in wsj:
        line = rem_tag(line)
        wsj_norm_corpus += line.replace(',','').split(' .')


In [235]:
print("WSJ:", wsj_corpus[5])
print("WSJ NORM: ",wsj_norm_corpus[5])


WSJ: Net for the quarter ended Sept. 30 fell to $ 150.8 million  or $ 1.20 a share  from $ 223 million  or $ 1.71 a share  a year ago
WSJ NORM:  Net for the quarter ended <NNP> <CD> fell to $ <CD> <CD>  or $ <CD> a share  from $ <CD> <CD>  or $ <CD> a share  a year ago


### Maximum Likelihood
Now, start by producing code to compute maximum-likelihood estimate probabilities. Your code should be configurable with respect to the n-gram order- i.e., you should be able to set it to compute bigram, trigram, 4-gram, etc. probabilities. Refer to J&M and the lecture slides for definitions as needed. If you would like to write your own n-gram tokenization code, feel free to do so, but you may also use the ngrams.py utility class which contains a routine to take a list of tokens and produce a stream of n-grams with appropriate padding for the start and end of sentences.

#### Tip: 
* Start with a very small "toy" corpus of just a couple of sentences for debugging. 

* As discussed in class, I strongly recommend using nested defaultdicts as the foundational data structure for your language model, where the "outer" key is the prefix, and the value retrieved by that prefix is a second defaultdict  containing possible suffices for that prefix, each of which is an "inner" key. E.g., p("TRUTHS" | "HOLD THESE") would be retrieved by first looking up "HOLD THESE" and then from the resulting dictionary, looking up "TRUTHS": prob = trigrams[("HOLD","THESE")]["TRUTHS"] . Note that this arrangement makes it very easy to e.g. find out the number of times a given history occurs, the total probability mass assigned to all of a history's continuations, etc., all of which will be extremely helpful in the next part of the assignment.

* Use tuples to represent prefixes. E.g., instead of the string "HOLD THESE", use the tuple ("HOLD", "THESE"). Note that, in Python, lists are mutable, and therefore may not be used as keys in dictionaries- but tuples are immutable, and so make excellent keys.

* Don't forget about numerical underflow issues! You'll want to represent probabilities as negative base-2 log probabilities, and modify your arithmetic accordingly. I recommend experimenting with [the bitweight Python library](https://github.com/stevenbedrick/bitweight) (see its unit tests for example usage).
* 

#### What to turn in:
* your code 
* use your code to create a simple language model for small_corpus named as small_lm and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create language model for wsj.pos.gz named as wsj_lm

In [238]:
def log2(x):
    """Return -log2(x)"""
    return -log(x, 2)

def pow2(x):
    """Return 2^-x"""
    return pow(2,-x)

def make_dict(corpus, order):
    """Returns a nested defaultdict(defaultdict), formatted as follows:
        [prefix]:[suffix]:prefix/suffix count"""
    corpus_list = []
    corpus_dict = defaultdict(lambda: defaultdict(int))

    corpus_list += [corpus[i].lower().split() for i in range(len(corpus))] #corpus[i].lower().split() ?
    chunks = ngrams(corpus_list, order)
    lchunks = list(chunks)
    
    for i in range(len(lchunks)):
        corpus_dict[lchunks[i][0] ][lchunks[i][1]] += 1

    return corpus_dict

def mle(corp_dict, sentence):
    """Temp test function
    Input: dictionary(dictionary()): corp_dict, String: sentence
    Returns: ngrams as a list
    Order identified from dictionary used"""
    order = len(list(corp_dict.keys())[0]) + 1
    n = ngrams([sentence.split()], order)
    return list(n)

def mle100(corp_dict, sentence):
    """Multiply version of MLE"""
    acc = 1
    order = len(list(corp_dict.keys())[0]) + 1
    n = list(ngrams([sentence.split()], order))
    #print(n)
    for entry in n:
        total = 0
        for i in corp_dict[entry[0]]:
            total += corp_dict[entry[0]][i]
        #print(total)
        prob = corp_dict[entry[0]][entry[1]] / total
        acc = acc * prob
        #print("acc@", entry[0], ":", entry[1] ,acc)
    return acc

def mle_log(corp_dict, sentence):
    """Log version of MLE"""
    sentence = sentence.lower()
    acc = 0 #in log2
    order = len(list(corp_dict.keys())[0]) + 1
    n = list(ngrams([sentence.split()], order))
    for entry in n:
        total = 0
        for i in corp_dict[entry[0]]:
            total += corp_dict[entry[0]][i]
        #print("total",total)
        suffix = corp_dict[entry[0]][entry[1]]
        if (suffix == 0):
            return inf
        prob = log2(suffix) - log2(total)
        #print("PROB", prob)
        assert prob >= 0, "-log2(prob) not > 0" #since negative log(fraction)
        acc = acc + prob
        #print("acc@", entry[0], ":", entry[1] ,acc)
    return acc

def print_dict(corpus_dict):
    for key in sorted(corpus_dict.keys()):
        print(key)
        for key2 in sorted(corpus_dict[key].keys()):
            print("\t", key2, ":", corpus_dict[key][key2])

### Small Toy Test Case
small_lm = make_dict(small_corpus, 2)

mle1 = mle(small_lm, "we example few there")
print(mle1[1][0])
print(list(mle1) )

### MLE with multiplication
print("0 until smoothing:", mle100(corpus_dict, "start")) #0-probability case
print("mle100", mle100(small_lm, "we start here")) #non-zero probability case

### MLE with log
res1 = mle_log(small_lm, "start")
print("0 until smoothing: log: ", res1, "decimal:", pow2(res1)) #0-probability case
res2 = mle_log(small_lm, "we start here") #non-zero probability case
print("mle log: ", res2, "decimal prob:", pow2(res2))

res3 = mle_log(small_lm, "Why dont we end there")
print("'why dont we end there':mle log: ", res3, "decimal prob:", pow2(res3))

('we',)
[(('<S_0>',), 'we'), (('we',), 'example'), (('example',), 'few'), (('few',), 'there'), (('there',), '</S_0>')]
0 until smoothing: 0.0
mle100 0.025
0 until smoothing: log:  inf decimal: 0.0
mle log:  5.321928094887362 decimal prob: 0.025000000000000012
'why dont we end there':mle log:  3.321928094887362 decimal prob: 0.1


In [221]:
wsj_lm = make_dict(wsj_corpus, 2)
wsj_norm_lm = make_dict(wsj_norm_corpus, 2)

In [236]:
res = mle_log(wsj_lm, "In New York") #non-zero probability case
print("mle (negative) log: ", res, "decimal prob:", pow2(res))

res = mle_log(wsj_lm, "Pineapple") #zero probability case
print("mle (negative) log: ", res, "decimal prob:", pow2(res))

mle (negative) log:  15.192551517368237 decimal prob: 2.670461263068215e-05
mle (negative) log:  inf decimal prob: 0.0


<font color="red">Self assessment:</font>

### Smoothing

Once you’ve got an unsmoothed model working, move on to implementing Witten-Bell smoothing. Refer to the slides and J&M for details on how that ought to work.

#### Tip: 
* You can modify an already-populated defaultdict to change its default value (for example, to store a default backoff value for a particular history) by changing the object’s default_factory attribute. Consult the documentation for examples of how this works.
* As defined, W-B smoothing is highly recursive; you may find it more efficient to re-frame the algorithm in iterative terms.
* As in the previous section, start small.
* [This may offer you some help on how to implement Witten-Bell smoothing](http://www.ee.columbia.edu/~stanchen/e6884/labs/lab3/x207.html)


#### What to turn in:
* your code 
* use your code to create a simple smoothed language model based on small_lm  and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create a smoothed language model based on wsj_lm

In [286]:
#Default value usage test>Pass
d1 = defaultdict(int)
d1["apple"] = 5
print(d1[5])
d1.default_factory = lambda: 6
print(d1.default_factory(), d1[100], d1[5], d1["apple"]) #Result 6 6 0 5
#Note: d1[5] set to initial value 0 after first call
#Note: Use d1.default_factory() to call lambda function directly without creating new entry

d2 = defaultdict(lambda: defaultdict(int))
d2[1][2] = 15
d2[1].default_factory = lambda: 0.5
print(d2[1][2], d2[1][111], d2[2][2]) #Expected 15, 0.5, 0

0
6 6 0 5
15 0.5 0


In [316]:
n = 2
test_case = "let us"
models = [] #list of dict(dict(ints)); each entry is a 1-gram, 2-gram, ..., n-gram

############## Small Toy Test Case

#Initialize n-gram dictionaries from unigram to desired n-gram
for i in range(n):
    small_lm = make_dict(small_corpus, i+1)
    models.append(small_lm.copy())
#Verify
#print_dict(models[n-1])

#Build up lambda terms
for i in range(0,n):
    list_grams = list(ngrams([test_case.split()], i+1))
    #print(list_grams)
    print(list_grams[0])
    keys = models[i].keys()
    print(keys)
    for key in keys:
        inner_keys = models[i][key].keys()
        print(inner_keys)
        
        tokens = 0
        for ikey in inner_keys:
            tokens += models[i][key][ikey]

        types = 0
        types = len(inner_keys)
        
        print(tokens, types)
        lamb = tokens / (tokens+types)
        models[i][key].default_factory = lambda: lamb

        print("lambda for n =",(i+1),"key:", models[i][key].default_factory())




((), 'let')
dict_keys([()])
dict_keys(['other', 'an', 'docs', 'let', 'that', 'there', 'a', 'be', 'end', 'few', 'so', 'examples', 'can', 'start', 'here', 'example', 'tokens', 'dont', 'never', 'why', 'us', 'we', 'with', 'in', 'words'])
39 25
0.609375
(('<S_0>',), 'let')
dict_keys([('there',), ('words',), ('example',), ('so',), ('let',), ('that',), ('dont',), ('few',), ('we',), ('never',), ('tokens',), ('with',), ('here',), ('docs',), ('other',), ('examples',), ('an',), ('can',), ('us',), ('start',), ('<S_0>',), ('a',), ('end',), ('be',), ('why',), ('in',)])
dict_keys(['</S_0>'])
1 1
dict_keys(['that'])
1 1
dict_keys(['docs', 'with'])
2 2
dict_keys(['few'])
1 1
dict_keys(['us'])
1 1
dict_keys(['we'])
1 1
dict_keys(['we'])
2 1
dict_keys(['other', 'tokens'])
2 2
dict_keys(['start', 'end', 'never'])
4 3
dict_keys(['start'])
1 1
dict_keys(['</S_0>', 'can'])
2 2
dict_keys(['a', 'in', 'an', 'so'])
4 4
dict_keys(['</S_0>'])
1 1
dict_keys(['</S_0>'])
1 1
dict_keys(['examples'])
1 1
dict_keys(['</

<font color="red">Self assessment:</font>

### Evaluation via Perplexity
Explore the effects of n-gram order using perplexity. Perform ten-fold cross-validation on the WSJ corpus. On each iteration, this will give you a different 90/10 training/test split; train a smoothed language model on the 9 training sections, and compute the average per-token perplexity of the tenth section. The slides from the language modeling lecture give the equation for perplexity computation (as does J&M chapter 4); you'll need to modify the equation a bit, since we're using log-probabilities. 

Now, try this for unigram, bigram, trigram, and 4-gram models. 

#### What to turn in
* your cross-validation function. You are not suppose to use any cross-validation function from any module. You should implement it by yourself.
* your perplexity function
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj.pos.gz
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj-normalized.pos.gz.
* Answer following 2 questions: 
    * How does perplexity change as the model order size increases?
    * How does perplexity change as the data changed?

<font color="red">Self assessment:</font>